In [2]:
# 패키지 로드
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os, random

from scipy import sparse
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

import plotnine
from plotnine import *

In [4]:
# 경로 설정
data_path = './data'
saved_path = './code/saved'
output_path = './code/submission'

In [5]:
# 데이터 불러오기 
history_df = pd.read_csv(os.path.join(data_path, 'history_data.csv'), encoding='utf-8')
profile_df = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')
meta_df = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')

In [6]:
history_df

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,20220301115719,MKID003,15,NaN,Y,N
1,3,20220301115653,20220301115809,MKID003,16,NaN,Y,N
2,3,20220301115653,20220301115958,MKID003,17,NaN,Y,N
3,3,20220301115653,20220301120118,MKID003,18,NaN,Y,N
4,3,20220301115653,20220301120229,MKID003,19,NaN,Y,N
...,...,...,...,...,...,...,...,...
1005646,33032,20220427155091,20220427155668,MKID003,381,NaN,Y,N
1005647,33032,20220427155091,20220427155680,MKID003,381,NaN,Y,N
1005648,33032,20220427155839,20220427155810,MKID003,125,NaN,Y,N
1005649,33032,20220427155706,20220427155838,MKID003,125,NaN,Y,N


In [14]:
!pip list

Package                      Version
---------------------------- -------------------
absl-py                      0.15.0
aiohttp                      3.8.1
aiosignal                    1.2.0
appdirs                      1.4.4
APScheduler                  3.9.1
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
asteroid                     0.4.1
asteroid-filterbanks         0.4.0
asttokens                    2.0.5
astunparse                   1.6.3
async-timeout                4.0.1
attrs                        21.4.0
audioread                    2.1.9
backcall                     0.2.0
backports.zoneinfo           0.2.1
beautifulsoup4               4.11.1
black                        19.10b0
bleach                       4.1.0
blinker                      1.4
Bottleneck                   1.3.4
Brotli                       1.0.9
brotlipy                     0.7.0
cached-property              1.5.2
cachetools                   4.2.2
certifi                      2021.

In [11]:
history_df.profile_id.value_counts()

2794     18190
3847     14014
13450     8086
6155      5606
6107      5495
         ...  
31130        1
19531        1
5650         1
13839        1
19700        1
Name: profile_id, Length: 8311, dtype: int64

In [13]:
history_df.nunique()   # 8311 프로필

profile_id           8311
ss_id              117749
log_time           797725
act_target_dtl          1
album_id            20695
payment                40
continuous_play         2
short_trailer           2
dtype: int64

In [8]:
history_df.act_target_dtl.value_counts()   # 시청시작 밖에 없음

MKID003    1005651
Name: act_target_dtl, dtype: int64

- 세션 시간과 로그 타임 차이  
